# 神经网络与逻辑回归
---------

1. 神经网络为何会流行起来?
训练样本的规模越超大,预测效果越好,这时传统机器学习方法抵不上的
![avatar](../img/dl1.png)

### 一. 浅层神经网络
1. 两层神经网络 (单隐层神经网络)  
( 1 )神经网络的中间节点被称作”隐藏层”.   
( 2 )层数计算不包括输入的特征属性层, 但包括输出层  
( 3 )每层神经网络的计算结果,用${ a }_{ j }^{ [i] }$表示, 其中$i$表示当前处在第几层网络中 (输入特征层算第0层)  
&nbsp;&nbsp;&nbsp;&nbsp;下标$j$表示该层的第几个节点输出的结果. $a$表示active激活函数  
![avatar](../img/lesslayer.png)

# 正则化及优化
-----------
### 一. 深度学习的使用层面
1. 训练,开发(验证),测试集的分配比例  
( 1 )小规模数据量情况下: 70% 20% 10%  
( 2 )大数据下: 98% 1% 1%  
( 3 )保证验证集和测试集来自同一个数据集, 这样才能对分布进行无偏估计  
2. 偏差与方差    
( 1 )偏差 : 训练集上的表现  
( 2 )方差: 验证集上的表现  
( 3 )高偏差合高方差时的选择  
![avatar](../img/1-highbiasandvariance.png)

### 二. 正则化
1. L2正则化  
带正则化的神经网络优化函数: $J({ w }^{ [1] },{ b }^{ [1] },{ w }^{ [2] },{ b }^{ [2] }...{ w }^{ [L] },{ b }^{ [L] })=\frac { 1 }{ m } \sum _{ i=1 }^{ m }{ L(\widehat { { y }^{ (i) } } ,{ y }^{ (i) }) } +\frac { \lambda  }{ 2m } \sum _{ l=1 }^{ L }{ { \parallel W\parallel  }_{ F }^{ 2 } } $  
其中, ${ \parallel W\parallel  }_{ F }^{ 2 }$称作Frobnius范数, 角标$F$代表Frobnius  
2. 数据扩增正则化  
很多情况下, 我们的样本不足. 比如计算机视觉里需要大量的输入样本进行学习(因为图片上的像素点有很多). 此时我们通过裁剪图片, 扭曲图片, 翻转图片来扩大测试集. 由于这些扩大的样本实在已有样本基础上加工而来, 而不是全新的训练样本, 因此可以起到正则化的作用
![avatar](../img/1-expendimg.png)  
3. Early Stoping  
Early Stoping意为提早结束算法学习. 每次完成一轮梯度下降后, 记录该算法在测试集和训练集上损失函数的值, 绘制曲线后发现, 通常训练集上的损失函数值会一路下降, 验证集上的函数值呈现先下降后上升的情况, 在验证集上损失函数值最低处停止学习. 

### 三.Dropout随机失活  
1. 随机失活的做法:  
( 1 )遍历神经网络的每一层, 并设置神经网络中每个节点消失的概率.(每个节点都以抛硬币的方式设置概率, 因此每个节点得以保留的概率为0.5), 此后我们随机删除一些节点, 并删除从该节点进出的连线, 最后得到一个节点更少, 规模更小的神经网络. 反向传播时, 就在这个被简化的网络上进行. 对于不同的训练样本, 可以重新选择失活的节点  
( 2 )随机失活可有效解决过拟合问题, 因为每个样本的训练, 都是在训练一个小得多的网络  
2. 反向随机失活 - inverted dropout  
假设在第3层上进行随机失活, 这里只演示了前向传播中的dropout, 反向传播同理  
( 1 ) keep_prob:节点得以保存的概率  
( 2 ) d3:第三层的dropout向量  
( 3 ) a3:第三层隐藏单元的激活函数, 经编码后, a3向量中为0的对应该节点已被dropout   
```python
# 设置每个节点得以保留的概率最多为0.8
keepprob = 0.8
# 生成一个与a3相同规模的dropout失活矩阵
d3 = np.random.rand(a3.shape[0],a3.shape[1]) < keepprob
# 还是原来一样计算正向传播,只是在结果上乘以节点存活的概率, 来模拟节点失活
a3 = np.multiply(a3,d3)
# 由于我们只保留了80%的节点,因此最终结果还要除以80%,保证a3的结果不变
a3 = a3/0.8
```
3. Dropout的直观理解  
( 1 )对每个隐藏单元来讲, 他们的每个输入都可能会随机消失. 直观上好像在说, 因为这些输入可能随机删除, 我们就不会对某个输入加上过大的权重(W向量), 如此来处理过拟合  
( 2 )每层的keep_prob可以设置为不同值.如下图, 第2层隐藏层的系数矩阵${ W }^{ [2] }\in { R }^{ 7*7 }$, 该层产生过拟合的概率大, 则可以把该层的keep_prob设置小一些.keep_prob=1表示保留所有单元 (通常输入层和输出层的keep_prob=1)  
![avatar](../img/1-dropout.png) 
( 3 )通常情况, 都会先关闭dropout, 即把每层的keep_prob设置为1. 在确保每次迭代,cost function都在下降, 即代码正确运行后, 再打开dropout进行正则化

# 加速神经网络训练
-------------
### 一. 正则化输入
1. 属性正则化的做法  
更新样本的feature形式, $x=\frac { x-\mu  }{ { \sigma  }^{ 2 } } $,其中  
( 1 ) ${ { \mu  } }_{ i }=\frac { 1 }{ m } \sum _{ i=1 }^{ m }{ { x }_{ i } } $  
( 2 ) ${ \sigma  }_{ i }^{ 2 }=\sum _{ i=1 }^{ m }{ { { (x }_{ i } }-{ \mu  }_{ i })^{ 2 } } $    
2. 为什么正则化输入可以加快训练速度  
正则化输入, 使得所有属性的均值为0, 方差为1. 把所有属性正则化后, 使得代价函数变得更为对称(椭球变成球), 梯度下降时, 就可以选择稍大的学习率, 增加每次梯度下降的步长, 加速学习

### 二. 梯度消失与梯度爆炸
1. 梯度爆炸与梯度消失是如何发生的  
设有一个超多层的神经网络如下图所示. 为简化训练过程的描述, 假设每个隐藏层的激活函数g(z)=z,且b=0
![avatar](../img/1-tiduxiaoshi.png)
 前者导致反向传播时产生的梯度值很大, 即梯度爆炸. 后者导致梯度值很小, 接近于0, 造成梯度消失
2. 权重初始化选择  
我们从单层神经网络开始, 来探究不同激活函数下, 参数的选择方法. (多层神经网络的输入x变为${ a }^{ l-1 }$)  
![avatar](../img/1-singlenet.png)
( 1 ) sigmoid函数, 则选择系数$\frac { 1 }{ n } \Longrightarrow  { W }^{ [l] } = np.random.randn(shape)*np.sqrt(1.0/n[L-1])$   
( 2 ) Relu函数, 则选择系数$\frac { 2 }{ n } \Longrightarrow  { W }^{ [l] } = np.random.randn(shape)*np.sqrt(2.0/n[L-1])$  
( 3 ) tanh函数, 则选择系数$\frac { 1 }{ n }或\frac { 1 }{ { n }^{ l }*{ n }^{ l-1 } }$

### 三. 使用mini-batch梯度下降
1. 为什么使用mini-batch  
传统梯度下降要计算所有样本的一个下降值, 然后取其平均.但神经网络往往有巨大的训练集, 所以不能让梯度下降一次计算全部样本, 转而选择一个子集在还是那个面进行梯度下降
```python
m = 100000
batch = 1000
Repeat until convege{
    for i in range(0,m/batch):
        x_batch = x.iloc[:,i*batch:(i+1)*batch]
        y_batch = x.iloc[:,i*batch:(i+1)*batch]
        # 梯度下降
}
```
2. 典型的batch步长选择
通常选择${ 2 }^{ n }$作为batch的步长 , $eg:64,128,512,1024 ...$

### 四. Momentum梯度下降 - 动量梯度下降
1. 指数加权平均  
( 1 ) 统计中称作"指数加权移动平均". 计算当天的指数加权平均为对"前一天指数加权平均"再加权计算,加上当天值得一个加权.如用${ \theta  }_{ t }$表示当天的观测值,加权系数$\beta =0.9$,指数加权值为${ V }_{ t }=\beta { V }_{ t-1 }+(1-\beta ){ \theta  }_{ t }$,默认${ V }_{ 0 }=0$  
( 2 ) 如下图,展示了一段时间下记录的每天华氏温度记录数值${ \theta  }_{ t }$.蓝色点为温度的散点图. 红色曲线为权值$\beta=0.9$时的指数加权平均温度值与天数的对应关系  
$${ V }_{ 0 }=0\\ { V }_{ 1 }=\beta { V }_{ 0 }+(1-\beta ){ \theta  }_{ 1 }\\ { V }_{ 2 }=\beta { V }_{ 1 }+(1-\beta ){ \theta  }_{ 2 }\\ ...\\ { V }_{ t }=\beta { V }_{ t-1 }+(1-\beta ){ \theta  }_{ t }$$  ${ V }_{ t }$近似于计算了$\frac { 1 }{ 1-\beta  } $天的平均值. 即:$\beta$=0.9,近似于平均了10天的温度变化.而当$\beta$=0.98,则近似于平均了50天的温度.此时再做图得到了绿色曲线.我们注意到,绿色曲线明显比紫色曲线平滑, 因为当$\beta$值越大时,需要平均的天数越多,因此得到的曲线就越平滑.而当$\beta$=0.5时,之平均了两天的温度,得到黄色曲线及其曲折
<img src="../img/2-expotentional.png" width = "40%" height = "40%"  align=center />
( 3 ) 为什么如上形式的公式叫做指数加权平均?  
我们展开$\\ { V }_{ 100 }=0.1*{ \theta  }_{ 100 }+0.9{ V }_{ 99 }\\ =0.1{ *\theta  }_{ 100 }+0.9({ 0.1*\theta  }_{ 99 }+0.9{ V }_{ 99 })\\ =0.1{ *\theta  }_{ 100 }+0.9({ 0.1*\theta  }_{ 99 }+0.9(0.1*{ \theta  }_{ 98 }+0.9{ V }_{ 97 }))\\ =0.1{ *\theta  }_{ 100 }+0.9*{ 0.1*\theta  }_{ 99 }+0.9*0.9*{ 0.1*\theta  }_{ 98 }+...{ 0.9 }^{ 99 }*0.1*{ \theta  }_{ 1 }\quad \{ \quad \because { V }_{ 0 }=0,\therefore { V }_{ 1 }=(1-\beta ){ \theta  }_{ 1 }\quad \} \\ =0.1{ *\theta  }_{ 100 }+\sum _{ i=1 }^{ 99 }{ 0.1*{ 0.9 }^{ i }{ \theta  }_{ 100-i } } +{ 0.9 }^{ n-i }{ V }_{ i }\quad \quad (即前面的sum项越多,则截止的i越小,则后面的{ 0.9 }^{ n-i }系数越小)$  
我们同时发现, ${ 0.9 }^{ 10 }\simeq \frac { 1 }{ e } $,$\because { (1-\frac { 1 }{ \varepsilon  } ) }^{ \varepsilon  }=e,\quad 且{ (1-\varepsilon ) }^{ \frac { 1 }{ \varepsilon  }  }=e$  
( 4 ) 指数加权平均的偏差修正  
我们注意到,使用指数加权后plot出的图像,紫线在初始状态接近于0,这是因为${ V }_{ 1 }=\beta { V }_{ 0 }+(1-\beta ){ \theta  }_{ 1 }=0.9*0+0.1{ \theta  }_{ 1 }=+0.1{ \theta  }_{ 1 }$,其结果只是第一天观测值的0.1倍.为了修正这个问题及以后的指数甲醛问题,引入了偏差修正,让${ V }^{ t }=\frac { { V }^{ t } }{ 1-{ \beta  }^{ t } } \\ $,修正后从紫线变为绿色线  
 <img src="../img/2-biascorrection.png" width = "40%" height = "40%"  align=center />

2. 动量梯度下降-快于普通梯度下降的方法  
( 1 ) 计算梯度的指数加权平均, 用这个指数加权平均后的梯度值做下降.  
( 2 ) 动量梯度下降,实在每次迭代时,计算一个${V}_{dw}$,这个值为$dw$的指数加权平均, 同时有${V}_{dw}(t=0)=0$  
${ \quad  V }_{ dw }=\beta *{ V }_{ dw }+(1-\beta )*dw\quad \quad 指数加权平均\\ \quad   W=W-\alpha { V }_{ dw }\quad \quad \quad \quad \quad \quad \quad 梯度下降更新值$  
$\beta=0.9$时得到的效果最好.通常情况下,使用动量梯度下降,不许再进行$\frac { 1 }{ 1-{ \beta  }^{ t } } $的偏差修正

3. 动量梯度下降在损失函数上的表现  
我们把损失函数投影到平面后,形成同心椭圆.传统梯度下降的做法,在纵轴的摆动很大,反而在横轴,真正的下降方向摆动很小.  
而动量梯度下降,在采用指数加权平均化梯度后,在纵向的下降速度很快.


### 五.RMSprop算法
1. RMSprop:全称root mean squre prop算法,也可用来加速梯度下降  
2. 为了避免混淆,这里我们使用${ \beta  }_{ 2 }$来表示权值  
$on\quad iteration\quad t:\\ \quad \quad compute\quad dw\\ \quad \quad { S }_{ dw }={ \beta  }_{ 2 }{ *S }_{ dw }+(1-{ \beta  }_{ 2 }*{ { dw } }^{ 2 })\\ \quad \quad w:=w-\alpha *\frac { dw }{ \sqrt { { S }_{ dw }+\varepsilon  }  } \quad \quad \quad \quad \quad //此处为了避免分母为0,于是加上一个很小的正数\varepsilon ,这个数值是多少无所谓,只要够小即可\varepsilon ={ 10 }^{ -8 }\\ $

### 六. Adam优化算法
1. Adam算法结合了momentum梯度下降和RMSprop梯度下降.其算法流程如下:  
$初始化:{ V }_{ dw }=0,{ S }_{ dw }=0\quad \\ Repeat:\\ \quad \quad \quad 计算dw,使用mini-batch梯度下降\\ \quad\quad \quad dw的动量:\quad { V }_{ dw }={ \beta  }_{ 1 }*{ V }_{ dw }+(1-{ \beta  }_{ 1 })dw\\ \quad \quad \quad dw的RMSprop值:\quad S_{ dw }={ \beta  }_{ 2 }*{ S }_{ dw }+(1-{ \beta  }_{ 2 }){ dw }^{ 2 }\\ \\ \quad \quad \quad 修正momentum:\quad { V }_{ dw }^{ corrected }=\frac { { V }_{ dw } }{ { { (1-{ \beta  }_{ 1 }) } }^{ t } } \\ \quad \quad \quad 修正RMSprop:\quad \quad { S }_{ dw }^{ corrected }=\frac { S_{ dw } }{ { (1-{ \beta  }_{ 2 }) }^{ t } } \\ \\ \qquad W:=W-\alpha \frac { { V }_{ dw }^{ corrected } }{ \sqrt { { S }_{ dw }^{ corrected }+\varepsilon  }  } $
2. Adam的默认超参数  
( 1 ) ${ \beta  }_{ 1 }=0.9$  
( 2 ) ${ { \beta  }_{ 2 } }=0.99$  
( 3 ) $\varepsilon ={ 10 }^{ -8 }$

### 七. 随时间慢慢减少学习率
1. 加速算法学习收敛速度的一类方法,就是随时间慢慢减少学习率. 我们称之为学习率衰减. 因为如果保持学习率不变,在算法接近收敛时,会在最优解附近来回震荡,使得不能快速收敛  
2. 学习率衰减的做法:  
我们定义$1epoch为,mini-batch的一轮计算(在某一段数据子集上)$  
( 1 ) 第一个做法: $\alpha :=\frac { 1 }{ 1\quad +\quad decacy-rate*epoch-num } *{ \alpha  }_{ 0 }$ , 其中${ \alpha  }_{ 0 }$为设置的学习率初始值,衰减率$decacy-rate$可设为1   
( 2 ) 第二个做法: 指数下降学习率$\alpha :={ 0.95 }^{ epoch }*{ \alpha  }_{ 0 }$  

### 八. 深度学习的局部最优解问题
深度学习中,很少可能出现局部最优解.因为"鞍点"是所有方向变化率为0的点.而这种条件,除了全局最优解外,很小可能出现在高维空间的局部最优解.因为大多数情况下,都是损失函数都是某些特征的凸函数,而对另一些特征则是凹函数

# 超参数调试
-------------
## 一. 超参数调试
#### 1. 深度学习中,我们往往需要调试很多的超参数,按照通常的重要比例做出如下排序(大->小)  
( 1 ) 学习率$\alpha $  
( 2 ) momentum动量$\beta $, 隐藏单元个数, mini-batch的size  
( 3 ) 神经网络层数,学习率的衰减率
#### 2. 使用随机取样,而不是网格搜索  
当我们确定了超参数的范围后,就可以对多为feature,在高维空间的多维体中随机选择参数,经过使用这些不同的超参数组合训练后,大致确定表现好的更小参数范围,再在这个范围内随机选择超参数组合训练,知道选择一个满意的超参数组合.  
如下所示,左侧为网格定点选择(不要这样),右侧为多维体的随机选择  
 <img src="../img/3-hyperparameter.png" width = "60%" height = "60%"/>
#### 3. 对数尺度取样  
很多情况下,我们并不能采取超参数整个取值范围内的平均随机采样.比如学习率$\alpha\in (0,1) $,就不能采取平均随机取样,因为学习率在0.01和0.02上区别不大,但是在0.01和0.001上的区别就很大,因此需要我们在每一小段上的学习率平均取值.此时我们可以划分为`(0.0001,0.001)`,`(0.001,0.01)`,`(0.01,0.1)`,`(0.1,1)`四个小段上平均取样.python实现如下
```python
>>> r = -4 * np.random.rand()
>>> rate = np.power(10,r)
>>> rate
0.0012504461026631186
```
我们遇到的另一个棘手问题是,指数平均参数$\beta$的选择. 因为$\beta$表示平均了$\frac { 1 }{ 1-\beta  } $天的数据,所以$\beta$越靠近1,指数平均的影响越大.  
$\beta =0.9\quad \quad ...\quad 0.999\\ 平均\quad 10\quad \quad ...\quad 1000\\ 1-\beta =0.1\quad \quad ...\quad 0.001$  
如此,把问题转化为和上面一致的结果  
我们现在把超参数的调试方式分为两类:"熊猫Panda"和"鱼子酱caviare".熊猫一次只产一子,意思为只照料一个模型,不断的取微调参数,每次微调都要耗费几周时间查看训练结果.相反鱼子酱模式则是一次训练多个模型,可以同时绘制出模型与参数的对应曲线.  

## 二. Batch Nomalization
#### 1. 正则化网络激活函数  
逻辑回归中,正则化输入可以加快学习,因为(-1,1)范围内的x在sigmoid函数上能更快收敛.我们计算$\mu =\frac { 1 }{ m } \sum { X } ;{ \sigma  }^{ 2 }=\frac { 1 }{ m } \sum { (X-\mu ) } { (X-\mu ) }^{ T };$归一化输入矩阵$ X:=\frac { X-\mu  }{ \sigma  } $.而在神经网络中,我们能否通过归一化上一隐藏层的输出,来加速下一层隐藏层的学习速度呢?即归一化${ a }^{ [l-1] }$.有大量研究表明,我们归一化${ z}^{ [l-1] }$来加速神经网络的学习速度.过程如下:    
( 1 ) 归一化z的过程:  
${ \mu  }=\frac { 1 }{ m } \sum { { z }^{ [i] } } \quad \quad \quad 计算z的均值\\ \\ { \sigma  }^{ 2 }=\frac { 1 }{ m } \sum { ({ z }^{ [i] }-{ \mu  }) } { ({ z }^{ [i] }-{ \mu  }) }^{ T }\quad \quad  计算z的方差\\ { z }_{ norm }^{ [i] }=\frac { { z }^{ i }-\mu  }{ \sqrt { { \sigma  }^{ 2 }+\varepsilon  }  } \quad \quad \quad \quad \quad \quad \quad \quad 归一化z\\ { \widetilde { z }  }^{ (i) }:=\gamma { z }_{ norm }+\beta \quad \quad \quad \quad \quad \quad \quad 通过指定\gamma 和\beta ,把z扩展到不同均值和方差的分布$   
( 2 ) 我们观察到,最后一步, 如果我们取$\gamma =\sqrt { { \sigma  }^{ 2 }+\varepsilon  } ,\beta =\mu $,则 ${ \widetilde { z }  }^{ (i) }={ z }^{ (i) }$,所以可以通过$\gamma 和\beta $的取值,把z变换成不同均值和方差的分布  

#### 2. BatchNorm与mini-batch混用加速神经网络训练  
使用BatchNorm对${ z }^{ i }$正则化后, 因为需要增加参数$\gamma 和\beta $对$z$变换均值和方差形成$\widetilde { z }$, 因此整个训练过程中又增加了2个学习参数$\gamma 和\beta $.   
此时, 学习参数为$W,\gamma,\beta $, 因此反向传播要计算$d{w}^{l}, d{\gamma}^{l}, d{\beta}^{l}$  

#### 3. Batchnorm有正则化作用
Batchnorm由于指定了$z$的均值和方差, 使得隐藏层呢过引入了额外的噪声, 因为不论反向传播如何, $z$总是符合均值0,方差1.这种引入噪声的思路, 就像Drioout中制定一个随机失活概率一样, 使得隐藏层的激活函数变得不敏感 

## 三. Softmax回归
#### 1. softmax激活函数  
( 1 )logistic激活函数输出了二分类下,每个分类的概率.而softmax回归,把激活函数推广到多个分类,而不仅仅是两类. 因此softmax回归的输出y是一个列向量,第i个元素代表获得第i分类的概率.即$y=\begin{bmatrix} { y }_{ 1 } \\ { y }_{ 2 } \\ ... \\ { y }_{ k } \end{bmatrix}$ .k为分类个数   
( 2 )多层神经网络中,前面几个隐藏层可以向logistic回归一样,只是最后的激活函数输出层改成使用softmax激活函数:${ y }_{ i }=\frac { exp\left( { a }_{ i }^{ \left[ l-1 \right]  } \right)  }{ \sum _{ i=1 }^{ k }{ exp\left( { a }_{ i }^{ \left[ l-1 \right]  } \right)  }  }  $.其中${ y }_{ i }$表示被预测为第$i$个分类的概率, 其中${ a }_{ i }^{ \left[ l-1 \right]  }$表示第$l-1$层的第$i$个隐藏节点输出值.  
( 3 ) 如下图所示例子,最后一层隐藏层有4个隐藏节点,表示为4分类问题.这四个隐藏节点每一个会有输出值${ a }_{ i }^{ \left[ l-1 \right]  }$. 则最终softmax得到的概率向量为  
```python
np.exp(a_n-1)/num.sum(a_n-1,axis=1) #a_n-1为上一层的输出
```
 <img src="../img/1-softmax.png" width = "40%" height = "40%"/>
 #### 2. 为什么softmax函数为${ y }_{ i }=\frac { exp\left( { a }_{ i }^{ \left[ l-1 \right]  } \right)  }{ \sum _{ i=1 }^{ k }{ exp\left( { a }_{ i }^{ \left[ l-1 \right]  } \right)  }  }  $   
( 1 ) 指数分布族的概念:  
指数分布族是有如下形式的概率密度函数的分布 : $p(y;\eta )=b(y)exp\left( { \eta  }^{ T }T(y)-a(\eta ) \right) $,其中有  
$\begin{cases} \eta :自然参数 \\ T(y):充分统计量,一般有T(y)=y \\ a(\eta ):log分区函数 \end{cases}$  
( 2 ) 现假设softmax服从指数分布族的形式,且设$T{ (y) }_{ i }=I\left\{ y=i \right\} $,即$T{ (y) }$向量的第$i$个元素的值, 用指示函数给出  
然后我们用${ \phi  }_{ i }$表示把样本预测为第$i$类的概率.则有 $p(y)={ { \phi  }_{ 1 } }^{ I\{ y=1\}  }{ { \phi  }_{ 2 } }^{ I\{ y=2\}  }...{ { \phi  }_{ k } }^{ I\{ y=k\}  }={ { \phi  }_{ 1 } }^{ T{ (y) }_{ 1 } }{ { \phi  }_{ 2 } }^{ T{ (y) }_{ 2 } }...{ { \phi  }_{ k } }^{ T{ (y) }_{ k } }$, 我们最大化对数似然得到  
$argmax\left( log(p(y)) \right) =exp\left[ T{ (y) }_{ 1 }log{ { \phi  }_{ 1 } }+T{ (y) }_{ 2 }log{ { \phi  }_{ 2 }...T{ (y) }_{ k }log{ { \phi  }_{ k } } } \right] \\ \quad \quad \quad \quad  \quad \quad \quad  =exp\left[ T{ (y) }_{ 1 }log{ { \phi  }_{ 1 } }+T{ (y) }_{ 2 }log{ { \phi  }_{ 2 }...(1-\sum _{ i=1 }^{ k-1 }{ T{ (y) }_{ i } } )log{ { \phi  }_{ k } } } \right] \\ \quad \quad \quad \quad \quad \quad \quad   =exp\left[ T{ (y) }_{ 1 }log{ \frac { { \phi  }_{ 1 } }{ { \phi  }_{ k } }  }+T{ (y) }_{ 2 }log\frac { { \phi  }_{ 2 } }{ { \phi  }_{ k } } ...T{ (y) }_{ k-1 }log\frac { { \phi  }_{ k-1 } }{ { \phi  }_{ k } } +log{ { \phi  }_{ k } } \right] \\ 对比b(y)exp\left( { \eta  }^{ T }T(y)-a(\eta ) \right) ,有\\ \begin{cases} b(y)=1 \\ a(\eta )=-log{ { \phi  }_{ k } } \\ \eta =\begin{bmatrix} log{ \frac { { \phi  }_{ 1 } }{ { \phi  }_{ k } }  } \\ log{ \frac { { \phi  }_{ 2 } }{ { \phi  }_{ k } }  } \\ ... \\ log{ \frac { { \phi  }_{ k-1 } }{ { \phi  }_{ k } }  } \end{bmatrix} \end{cases}\Longrightarrow { \eta  }_{ i }=log(\frac { { \phi  }_{ i } }{ { \phi  }_{ k } } )\Longrightarrow { \phi  }_{ i }={ e }^{ { \eta  }_{ i } }{ \phi  }_{ k }\Longrightarrow \because \sum _{ i=1 }^{ k }{ { \phi  }_{ i } } =1\quad \therefore \sum _{ i=1 }^{ k }{ { e }^{ { \eta  }_{ i } }{ \phi  }_{ k } } =1\Longrightarrow { \phi  }_{ k }=\frac { 1 }{ \sum _{ i=1 }^{ k }{ { e }^{ { \eta  }_{ i } } }  } 将该式带回{ \phi  }_{ i }={ e }^{ { \eta  }_{ i } }{ \phi  }_{ k },有{ \phi  }_{ i }=\frac { { e }^{ { \eta  }_{ i } } }{ \sum _{ i=1 }^{ k }{ { e }^{ { \eta  }_{ i } } }  } $   
#### 3. Softmax反向传播
( 1 ) softmax损失函数 : 
$L(\widehat { y } ,y)=\sum _{ i=1 }^{ m }{ \left[ -\sum _{ j=1 }^{ k }{ { y }_{ j }\log { \widehat { { y }_{ j } }  }  }  \right]  } $   
( 2 ) 这个损失函数类比于logistic regresion的损失函数, 其中$k$为分类个数
